In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

from matplotlib import rc
rc('font',family='AppleGothic')

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
from sklearn import linear_model
from scipy import stats
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import glob
import os

import time
import re


In [94]:
### 외국영화 예산 데이터

international_film = pd.read_csv('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/International_film_budget_sales.csv')
international_film

,Title,Budget,Domestic,International
0,Avengers: Endgame,"$400,000,000","$858,373,000","$2,797,800,564"
1,Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,071,802","$1,045,713,802"
2,Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,395,316,979"
3,Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,064,615,817"
4,Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,359,754"
...,...,...,...,...
195,Moonraker,"$31,000,000","$70,300,000","$210,300,000"
196,The Grand Budapest Hotel,"$31,000,000","$59,076,019","$163,503,514"
197,Vacation,"$31,000,000","$58,884,188","$102,306,500"
198,Arachnophobia,"$31,000,000","$53,208,180","$53,208,180"


In [95]:
### 필요없는 열 제거

international_film.drop(['Domestic'],axis=1,inplace=True)

In [96]:
### 숫자 float 형태로 변환 후 $ -> * 환율 = 원화

international_film['Budget'] = international_film['Budget'].str.replace('$','')
international_film['International'] = international_film['International'].str.replace('$','')

international_film['Budget'] = international_film['Budget'].str.replace(',','').astype('float')
international_film['International'] = international_film['International'].str.replace(',','').astype('float')

international_film['Budget'] = international_film['Budget']*1200
international_film['International'] = international_film['International']*1200

/Users/hanjaemin/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/hanjaemin/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.


In [131]:
### 열이름 변경

international_film.rename(columns = {'Title' : 'moviename', 'Budget' : 'budget', 'International' : 'sales'}, 
                          inplace = True)

### 열순서 변경

international_film = international_film.reindex(columns=['moviename', 'sales', 'budget'])

In [174]:
### 매출액 순으로 정렬 후 익덱스 재설정

international_film.sort_values(by=['sales'],ascending=False, inplace=True)
international_film.reset_index(drop=True, inplace = True)
international_film

,moviename,sales,budget
0,Avatar,3.415079e+12,2.844000e+11
1,Avengers: Endgame,3.357361e+12,4.800000e+11
2,Titanic,2.649584e+12,2.400000e+11
3,Star Wars Ep. VII: The Force Awakens,2.477539e+12,3.672000e+11
4,Avengers: Infinity War,2.458032e+12,3.600000e+11
...,...,...,...
195,The Young and Prodigious T.S. Spivet,4.292244e+08,3.960000e+10
196,Freaks of Nature,8.514960e+07,3.960000e+10
197,George and the Dragon,0.000000e+00,3.840000e+10
198,The Batman,0.000000e+00,1.200000e+11


In [142]:
international_film.to_csv('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Budget_Data/Preprocessing_Data/International_Data.csv',index = False)
international_film.to_excel('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Budget_Data/Preprocessing_Data/International_Data.xlsx',index = False)

In [206]:
### 한국영화 예산 데이터

korea_film = pd.read_csv('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Budget_Data/Korean_movie_budget - Sheet1.csv',encoding='euc-kr')
korea_film

,title,budget,audiences,domestic_sales
0,대호,1.700000e+10,1584168.0,1.210481e+10
1,좋은놈 나쁜놈 이상한놈,1.740000e+10,6684933.0,4.374755e+10
2,암살,1.800000e+10,12705700.0,9.846313e+10
3,태풍,2.000000e+10,3035086.0,1.937256e+10
4,미스터고,2.200000e+10,1328888.0,9.337976e+09
5,마이웨이,2.800000e+10,1586042.0,1.181384e+10
6,디워,3.000000e+10,7855441.0,4.933993e+10
7,군함도,3.300000e+10,6592151.0,5.050000e+10
8,신과함께,4.000000e+10,12274996.0,1.026661e+11
9,설국열차,4.370000e+10,9349993.0,6.701009e+10


In [207]:
korea_film.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           30 non-null     object 
 1   budget          30 non-null     float64
 2   audiences       30 non-null     float64
 3   domestic_sales  30 non-null     float64
dtypes: float64(3), object(1)
memory usage: 1.1+ KB


In [208]:
### 열이름 변경 및 재정렬

korea_film.rename(columns = {'title' : 'moviename', 'audiences' : 'attendance' , 'domestic_sales' : 'sales'}, inplace = True)
korea_film = korea_film.reindex(columns = ['moviename', 'sales', 'attendance', 'budget'])

In [226]:
### 매출액 내림차순 정렬

korea_film.sort_values(by=['sales'],ascending=False,inplace=True)
korea_film.reset_index(drop=True, inplace=True)
korea_film

,moviename,sales,attendance,budget
0,신과함께,1.026661e+11,12274996.0,4.000000e+10
1,암살,9.846313e+10,12705700.0,1.800000e+10
2,도둑들,9.366481e+10,12983178.0,1.150000e+10
3,설국열차,6.701009e+10,9349993.0,4.370000e+10
4,해적,6.637068e+10,8666046.0,1.340000e+10
5,베를린,5.235557e+10,7166290.0,1.100000e+10
6,군함도,5.050000e+10,6592151.0,3.300000e+10
7,디워,4.933993e+10,7855441.0,3.000000e+10
8,완벽한 타인,4.435081e+10,5293435.0,3.800000e+09
9,좋은놈 나쁜놈 이상한놈,4.374755e+10,6684933.0,1.740000e+10


In [224]:
korea_film.to_excel('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Budget_Data/Preprocessing_Data/Korea_Data.xlsx',index=False)
korea_film.to_csv('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Budget_Data/Preprocessing_Data/Korea_Data.csv',index=False)